### Useful example:
https://www.geeksforgeeks.org/3d-curve-fitting-with-python/

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # interactive 3D plots
from scipy.optimize import curve_fit # non-linear fit

In [61]:
data = pd.read_csv('CK_vs_entropy_data_scan.csv')
# Remove running time and seed
data = data.drop('elapsed_time', axis=1)
data = data.drop('seed', axis=1)
#data.head()

In [62]:
# average over network topologies with same n, r, and eta
averaged_data = pd.DataFrame(data.groupby(['n','r','eta'])['ck_mean_size'].mean())
averaged_data = averaged_data.reset_index()
# Add log2(r) and difference
averaged_data['log2(r)'] = np.log2(averaged_data['r'])
averaged_data['difference'] = averaged_data['ck_mean_size'] - averaged_data['log2(r)']
#averaged_data.head()

In [86]:
n = 11

df = averaged_data[averaged_data.n == n]
df.head()

,n,r,eta,ck_mean_size,log2(r),difference
1763,11,2,0.006075,2.55,1.0,1.55
1764,11,2,0.015903,2.35,1.0,1.35
1765,11,2,0.024705,2.35,1.0,1.35
1766,11,2,0.036875,2.20,1.0,1.20
1767,11,2,0.044518,2.25,1.0,1.25


In [87]:
#fig = px.scatter_3d(df, x='r', y='eta', z='difference')
#fig.update_layout(scene=dict(aspectmode='cube'))
#fig.update_traces(marker={'size': 3}) 
#fig.show()

### Fit

In [88]:
def func(r_eta, A, B, r0, eta0):
    # it needs to be function of a single variable
    # one way around is to have r_eta = (r,eta)
    r, eta = r_eta
    return A + B/((r+r0)*(eta+eta0)**2)

In [89]:
r_data = list(df.r)
eta_data = list(df.eta)
difference_data = list(df.difference)

In [90]:
# Perform curve fitting
# It doesn't converge without a starting point
popt, pcov = curve_fit(func, (r_data, eta_data), difference_data, p0 = [1,100,10,1])
# Print optimized parameters
print(popt)

[-1.11866603e+00  7.37089911e+03  7.67028564e+01  6.48314874e+00]


### Plot

In [91]:
import plotly.graph_objs as go

In [92]:
r_range = np.linspace(0, 100, 101)
eta_range = np.linspace(0, 10,11)
x, y = np.meshgrid(r_range, eta_range)
z = func((x, y), *popt)

In [93]:
fig = px.scatter_3d(df, x='r', y='eta', z='difference')
fig.update_layout(scene=dict(aspectmode='cube'))

surface_trace = go.Surface(z=z, x=x, y=y, showscale=False, opacity=0.5, colorscale='gray')
fig.update_traces(marker={'size': 4}) 

# Add the surface plot trace to the existing figure
fig.add_trace(surface_trace)
fig.show()